In [1]:
## STEP 1: Set up the workspace.

import json

#GETTING MY ID:
id = open('my_id.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]

from azureml.core import Workspace

ws = Workspace.create(name="ModifiedDate_Prediction",
                      subscription_id = my_id,
                      resource_group = "GRUPOSQL_3",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-37d4465ee2
Deploying KeyVault with name modifiedkeyvaultc54c33fa.
Deploying StorageAccount with name modifiedstorage1ab6dad93.
Deploying AppInsights with name modifiedinsights6c986445.
Deployed AppInsights with name modifiedinsights6c986445. Took 40.89 seconds.
Deploying Workspace with name ModifiedDate_Prediction.
Deployed Workspace with name ModifiedDate_Prediction. Took 35.18 seconds.
Registering model model


In [2]:
################ score.py ###################
import json

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)



def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)
    
    #Realizar prediccion
    result = model.predict(data).tolist()

    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [3]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn'])


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name='banruptcy-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\Iván Ortiz\AppData\Local\Temp\ipykernel_48304\2758913308.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [4]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"


In [5]:
print(scoring_uri)

http://452dace3-ec7b-4efe-8b09-0dbf3bfc4e2e.centralindia.azurecontainer.io/score
